In [1]:
%pip install requests beautifulsoup4 pyspark

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\renato.valentim\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pyspark.sql.functions as f
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Crawler PySpark").getOrCreate()

In [5]:
MAIN_PATH = "C:/Users/renato.valentim/Documents/ElasticSearch"
FILE_PATH = f"{MAIN_PATH}/urls_concorrencia.csv"

In [13]:
df = (
  spark.read
  .option("header", True)
  .option("inferSchema", True)
  .option("multiline", True)  # Mantém valores multilinha dentro das células
  .option("quote", '"')       # Garante que valores entre aspas sejam lidos corretamente
  .option("escape", "\\")     # Escape correto para caracteres especiais
  .option("sep", ",")         # Define o delimitador correto
  .option("ignoreLeadingWhiteSpace", True)  # Evita espaços em branco extras
  .option("ignoreTrailingWhiteSpace", True) # Evita espaços em branco no final
  .csv(FILE_PATH)
)

In [14]:
links_ofertas = [str(x.URLS) for x in df.collect()]

In [15]:
# Função para extrair os dados da nova página
def extrair_dados(url, id):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        resposta = requests.get(url, headers=headers)
        resposta.raise_for_status()
        
        soup = BeautifulSoup(resposta.text, 'html.parser')
        
        # Dicionário para armazenar todos os dados
        dados = {'id': id, 'url': url}
        
        # 1. Extrair nome e descrição
        secao_descricao = soup.find('div', class_='cav--c-eNhzRw-fiXSPN-sm-12')
        if secao_descricao:
            nome_produto = secao_descricao.find('span', class_='cav--c-gNPphv-hyvuql-weight-bold')
            if nome_produto:
                dados['nome'] = nome_produto.get_text(strip=True)
            else:
                dados['nome'] = 'N/A'
                
            descricao = secao_descricao.find('div', class_='cav--c-gNPphv-igBCBxe-css')
            if descricao:
                dados['descricao'] = descricao.get_text(strip=True)
            else:
                dados['descricao'] = 'N/A'
        else:
            print(f"Seção de descrição não encontrada em {url}")
            dados['nome'] = 'N/A'
            dados['descricao'] = 'N/A'
        
        # 2. Extrair tabelas de características
        tabelas = soup.find_all('table', class_='cav--c-gusfms-gjkaP-striped-odd')
        for tabela in tabelas:
            linhas = tabela.find_all('tr', class_='cav--c-PJLV')
            for linha in linhas:
                colunas = linha.find_all('td', class_='cav--c-difWUU')
                if len(colunas) == 2:
                    atributo = colunas[0].get_text(strip=True)
                    valor = colunas[1].get_text(strip=True)
                    # Tratar o caso do Manual de Montagem (extrair o link)
                    if atributo == 'Manual de Montagem':
                        link_manual = colunas[1].find('a')
                        valor = link_manual['href'] if link_manual else valor
                    dados[atributo] = valor
        
        # 3. Extrair o JSON-LD
        scripts = soup.find_all('script', type='application/ld+json')
        for script in scripts:
            try:
                json_data = json.loads(script.string)
                # Extrair breadcrumb
                if json_data.get('@type') == 'BreadcrumbList':
                    categorias = [item['item']['name'] for item in json_data.get('itemListElement', [])]
                    dados['caminho_categoria'] = ' > '.join(categorias)
                
                # Extrair dados do produto (se houver @type: "Product")
                if json_data.get('@type') == 'Product':
                    dados['productID'] = json_data.get('productID', 'N/A')
                    dados['sku'] = json_data.get('sku', 'N/A')
                    dados['imagem'] = json_data.get('image', 'N/A')
                    dados['marca_json'] = json_data.get('brand', {}).get('name', 'N/A')
                    dados['condicao'] = json_data.get('itemCondition', 'N/A')
                    if 'offers' in json_data and json_data['offers']:
                        oferta = json_data['offers'][0] if isinstance(json_data['offers'], list) else json_data['offers']
                        dados['preco'] = oferta.get('price', 'N/A')
                        dados['moeda'] = oferta.get('priceCurrency', 'N/A')
                        dados['disponibilidade'] = oferta.get('availability', 'N/A')
            except json.JSONDecodeError:
                continue
        
        return dados
    
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None

In [16]:
# Lista para armazenar os dados
dados_ofertas = []

# Processar os links com IDs sequenciais
for id, link in enumerate(links_ofertas, start=1):
    dados = extrair_dados(link, id)
    if dados:
        dados_ofertas.append(dados)
    time.sleep(1)  # Delay para evitar bloqueios

# Salvar em JSON
with open('caracteristicas_ofertas_madeira.json', 'w', encoding='utf-8') as arquivo_json:
    json.dump(dados_ofertas, arquivo_json, ensure_ascii=False, indent=4)

print("Dados salvos em 'caracteristicas_ofertas_madeira.json'")

Erro ao processar https://www.madeiramadeira.com.br/guarda-roupa-casal-com-espelho-3-portas-4-gavetas-188: 404 Client Error: Not Found for url: https://www.madeiramadeira.com.br/guarda-roupa-casal-com-espelho-3-portas-4-gavetas-188
Dados salvos em 'caracteristicas_ofertas_madeira.json'
